## [127\. Word Ladder](https://leetcode.com/problems/word-ladder/)

從 ```beginWord``` 走到 ```endWord```, 每次只能變換一個字, 回傳需要幾步才能走到目標


### BFS

可以把題目想像成一個 graph, 每個單字是個 node, 有相連的就是可以變過去的單字

針對 26 個字母做 BFS, 使用 queue 儲存下一個 node

time: $O(M * 26 * N)$, where M is the length of the single word, N is the length of wordList

space: $O(N)$

...

In [ ]:
from collections import deque, defaultdict

class Solution:
    def ladderLength(self, beginWord: str, endWord: str, wordList: List[str]) -> int:

        ## remove duplicate
        wordList = set(wordList)
        if endWord not in wordList or not wordList:
            return 0
        queue = deque([beginWord])
        step = 0
        M = len(beginWord)
        visited = set()
        chars  = set('abcdefghijklmnopqrstuvwxyz')

        while queue:
            step += 1
            for i in range(len(queue)):
                word = queue.popleft()
                for i in range(M):
                    for c in chars:
                        nextWord = word[:i] + c + word[i+1:]
                        if nextWord == endWord:
                            return step + 1
                        if nextWord not in visited and nextWrod in wordList:
                            visited.add(nextWord)
                            queue.append(nextWord)
        return 0

## [199. Binary Tree Right Side View](https://leetcode.com/problems/binary-tree-right-side-view/)

### BFS

使用 queue 實現 level order traversal

將每一層最右邊的元素 pop 出來, 放進 res 中

In [ ]:
class Solution:
    def rightSideView(self, root: TreeNode) -> List[int]:
        if not root:
            return []
        res = []
        queue = [root]
        # bfs
        while queue:
            for _ in range(len(queue)):
                node = queue.pop(0)
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
            res.append(node.val)
        return (res)

## [1197. Minimum Knight Moves](https://leetcode.com/problems/minimum-knight-moves/)

騎士最少需要幾步才能走到目標地點

### BFS

* (x, y), (-x, y), (x, -y), (-x, -y) 會有一樣的結果

總共有八個可能的方向

(2,1), (-2,1), (2,-1), (-2,-1)

(1,2), (-1,2), (1,-2), (-1,-2)

* 不需要 (-2, -1) 或 (-1, -2) 因為這兩個方向只會往同一個方向前進

符合條件:

-1 <= a+dx <= x+2

-1 <= b+dy <= y+2

因為不需要檢查 (abs(x), abs(y)) 以外的可能

time: O(|x| * |y|)

..

In [1]:
## BFS
from collections import deque
class Solution:
    def minKnightMoves(self, x: int, y: int) -> int:
        queue = deque([(0,0,0)])
        x, y = abs(x), abs(y)
        visited = set([(0,0)])
        while queue:
            a, b, step = queue.popleft()
            if (a,b) == (x,y):
                return step
            for dx, dy in [(1,2),(2,1),(1,-2),(2,-1),(-1,2),(-2,1)]:
                if (a+dx, b+dy) not in visited and (-1<=a+dx<=x+2) and (-1 <= b+dy <= y+2):
                    visited.add((a+dx, b+dy))
                    queue.append((a+dx, b+dy, step+1))
        return -1

### Bi-direction BFS

從兩個方向接近答案

使用兩個 queue 儲存下一個節點, 從起點出發的 qo, 跟從終點出發的 qt

使用兩個 hashmap 儲存從起點/終點到某點需要的路徑

time: idealy half of time: O(|x| * |y|) 

In [ ]:
## Bi-direction BFS
## time: idealy half of time: O(|x| * |y|) 
class Solution:
    def minKnightMoves(self, x: int, y: int) -> int:
        x, y = abs(x), abs(y)
        qo = deque([(0, 0, 0)])
        qt = deque([(x, y, 0)])
        do, dt = {(0,0): 0}, {(x,y): 0}
        while True:
            ox, oy, ostep = qo.popleft()
            if (ox, oy) in dt:
                return ostep + dt[(ox, oy)]
            tx, ty, tstep = qt.popleft()
            if (tx, ty) in do:
                return tstep + do[(tx, ty)]
            for dx, dy in [(1,2),(2,1),(1,-2),(2,-1),(-1,2),(-2,1),(-1,-2),(-2,-1)]:
                if (ox+dx, oy+dy) not in do and -1 <= ox+dx <= x+2 and -1 <= oy+dy <= y+2:
                    qo.append((ox+dx, oy+dy, ostep+1))
                    do[(ox+dx,oy+dy)] = ostep+1
                if (tx+dx, ty+dy) not in dt and -1 <= tx+dx <= x+2 and -1 <= ty+dy <= y+2:
                    qt.append((tx+dx, ty+dy, tstep+1))
                    dt[(tx+dx,ty+dy)] = tstep+1
        return -1

## DFS DP

Base case: (0,0), (1,1), (1,0), (0,1)

* 因為從這些點出發, 有可能跨出第一象限

time: $O(log(max(|x|, |y|)))$

...

In [ ]:
class Solution:
    def minKnightMoves(self, x: int, y: int) -> int:
        cache = {(0,0): 0, (1,1): 2, (1,0): 3, (0,1): 3}
        def dp(x,y):
            if (x,y) in cache:
                return cache[(x,y)]
            res = min(dp(abs(x-1),abs(y-2)), dp(abs(x-2),abs(y-1))) + 1
            cache[(x,y)] = res
            return res
        return dp(abs(x),abs(y))

## [994. Rotting Oranges](https://leetcode.com/problems/rotting-oranges/)

### BFS

Initialize:
1. put all rotten organes (2) into the queue
2. aggreate fresh oranges (1)

timestamp start from -1 because the answer start from 1

...

In [ ]:
class Solution:
    def orangesRotting(self, grid: List[List[int]]) -> int:
        ## bfs: start from [2] and spread to other oranges
        directions = [(1,0),(0,1),(-1,0),(0,-1)]
        queue = collections.deque()
        
        ## initialize: put 2 in queue, sum up 1
        fresh = 0
        for i in range(len(grid)):
            for j in range(len(grid[i])):
                if grid[i][j] == 2:
                    queue.append((i, j))
                if grid[i][j] == 1:
                    fresh += 1
        if fresh == 0:
            return 0
        step = -1
        while queue:
            step += 1
            for _ in range(len(queue)):
                (oi, oj) = queue.popleft()
                for (di, dj) in directions:
                    ni = oi+di
                    nj = oj+dj
                    if 0 <= ni < len(grid) and 0 <= nj < len(grid[0]):
                        if grid[ni][nj] == 1:
                            grid[ni][nj] = 2
                            queue.append((ni,nj))
                            fresh -= 1
        return step if fresh == 0 else -1

## [103. Binary Tree Zigzag Level Order Traversal](https://leetcode.com/problems/binary-tree-zigzag-level-order-traversal/)

### BFS

levle order traversal

odd level: append from left to right

even level: append from right to left

using deque in every level for faster append

...

In [ ]:
from collections import deque

class Solution:
    def zigzagLevelOrder(self, root: TreeNode) -> List[List[int]]:
        # queue: bfs
        # even level: from left to right, append level from right 
        # odd level: from right to left, append level from left
        if not root:
            return res
        res = []
        even = True
        queue = deque()
        queue.append(root)
        
        while queue:
            level = deque()
            for i in range(len(queue)):
                node = queue.popleft()
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
                if even:
                    level.append(node.val)
                else:
                    level.appendleft(node.val)
            even = not even
            res.append(level)
        
        return res

## [863. All Nodes Distance K in Binary Tree](https://leetcode.com/problems/all-nodes-distance-k-in-binary-tree/)

### BFS

1. transform tree to ConnectTable by using hashmap
2. queue initialize with the target
3. BFS neighbor nodes append to the queue if not visited yet
4. BFS until the distance reach K
5. return current queue which will contian all nodes distance K

...

In [ ]:
from collections import deque
from collections import defaultdict

class Solution:
    def distanceK(self, root: TreeNode, target: TreeNode, K: int) -> List[int]:
        def makeConnectTable(table, parent, node):
            if not node:
                return
            if parent:
                table[parent.val].append(node.val)
                table[node.val].append(parent.val)
            if node.left:
                makeConnectTable(table, node, node.left)
            if node.right:
                makeConnectTable(table, node, node.right)

        connectTable = defaultdict(list)
        makeConnectTable(connectTable, None, root)
        queue = deque([target.val])
        visited = set([target.val])
        currentDistance = 0
        
        while queue:
            if currentDistance == K:
                break
            for _ in range(len(queue)):
                node = queue.popleft()
                for neighbor in connectTable[node]:
                    if neighbor not in visited:
                        queue.append(neighbor)
                        visited.add(neighbor)
            currentDistance += 1
        
        return list(queue)

## [909. Snakes and Ladders](https://leetcode.com/problems/snakes-and-ladders/)

將 board 變成照順序的一維陣列 arr

反著遍歷 board (start from bottom-right): 
* even line: from right to left, remain original order
* odd line: from left to right, reverse order 

using queue for BFS

遍歷 cur + i

* 如果走到終點, 回傳目前的步數

* 如果碰到普通的格子(-1), 將該格加進 queue

* 如果碰到特別的格子, 將最後走到的格子加進 queue

...

In [ ]:
class Solution:
    def snakesAndLadders(self, board: List[List[int]]) -> int:
        arr=[0] ## place holder for 0-index
        for i,row in enumerate(board[::-1]):
            if i%2: ## odd line
                arr+=row[::-1]
            else: ## even line
                arr+=row
        
        t = len(board)*len(board)
        queue=collections.deque([1])
        visit=set([1])
        move=0
        while queue:
            for _ in range(len(queue)):
                cur = queue.popleft()
                if cur == t:
                    return move
                for i in range(1,7):
                    if cur+i<=t and cur+i not in visit:
                        visit.add(cur+i)
                        if arr[cur+i]==-1:
                            queue.append(cur+i)
                        else:
                            queue.append(arr[cur+i])
            move+=1
        return -1

## [207. Course Schedule](https://leetcode.com/problems/course-schedule/)

### Cycle Detection by DFS

dfs 遍歷 prerequisites 如果有 cycle 代表無法完成

使用三個 set 將每個節點分類

1. white set: 尚未 visit
2. grey set: 已經 visit, 但還有 neighbor 還沒 visit
3. black set: 所有 neighbor 都被 visit

檢查 neighbor 的狀態:

1. 該 neighbor 在 white set: 還沒 visit 過, 放進 grey set, 並 dfs 所有相鄰的 white 節點
2. 該 neighbor 在 grey set: 發現 cycle 回傳 False
3. 該 neighbor 在 black set: 已經 visit 過了, 不需要再次遍歷

time: $O(V + E)$

space: $O(V + E)$


In [ ]:
from collections import defaultdict
class Solution:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        ## initialize: put all nodes into white set
        white = defaultdict(list)
        for x, y in prerequisites:
            white[y].append(x)
        sorted(white.items())
        
        grey = set()
        black = set()
        
        def dfs(i):
            if i in black:
                return True
            if i in grey:
                return False
            else:
                grey.add(i)
                for j in white[i]:
                    if not dfs(j):
                        return False
                black.add(i)
                return True
        
        for i in range(numCourses):
            if not dfs(i):
                return False
        return True

## [210. Course Schedule II](https://leetcode.com/problems/course-schedule-ii/)

更進一步要回傳可行上課順序的 list

如果無法完成所有課程, 回傳 []

...

In [ ]:
class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        ## initialize: put all nodes into white set
        white = defaultdict(list)
        for x, y in prerequisites:
            white[y].append(x)
        sorted(white.items())
        
        grey = set()
        black = set()
        order = []


        def dfs(i):
            if i in black:
                return True
            if i in grey:
                return False
            else:
                grey.add(i)
                for j in white[i]:
                    if not dfs(j):
                        return False
                black.add(i)
                order.append(i)
                return True
        
        for i in range(numCourses):
            if not dfs(i):
                return []
        return order[::-1]